In [90]:
from glob import glob
from tqdm import tqdm
import json

In [91]:
rejected_words = [
    'kebutuhan',
    'berbeda',
    'bahwa',
    'Kode',
    'kode',
    'nomor',
    'RMXX,XXX',
    'kompleksitas',
    'listrik'
]
rejected_words = set(rejected_words)

In [92]:
files = sorted(glob('output/*.json'), key = lambda x: int(x.split('/')[-1].replace('.json', '')))
len(files)

51714

In [95]:
count, indon_count = 0, 0

indon_texts = []
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)

    if data['output'] is None:
        continue

    set_ins = set(data['instruction'].split())
    set_output = set(data['output'].split())

    if len(set_ins & rejected_words) or len(set_output & rejected_words):
        indon_texts.extend([data['instruction'], data['output']])

    count += 1

100%|██████████████████████████████████| 51714/51714 [00:01<00:00, 37496.60it/s]


In [96]:
indon_count / count

0.0

In [97]:
len(indon_texts)

9748

In [98]:
len(set(indon_texts))

9748

In [99]:
with open('instructions-indon.texts', 'w') as fopen:
    for t in set(indon_texts):
        fopen.write(f'{json.dumps(t)}\n')

In [100]:
!cp instructions-indon.texts ~/ssd3/ctranslate2

In [101]:
mapping = {}
with open('/home/husein/ssd3/ctranslate2/instructions-indon.texts.requested') as fopen:
    for l in fopen:
        l = json.loads(l)
        mapping[l['src']] = l['r']
        
len(mapping)

9748

In [104]:
count, indon_count = 0, 0

with open('augmented-alpaca_data_cleaned.jsonl', 'w') as fopen_l:
    for f in tqdm(files):
        with open(f) as fopen:
            data = json.load(fopen)
            
        if data['output'] is None:
            continue
        
        set_ins = set(data['instruction'].split())
        set_output = set(data['output'].split())
        
        if len(set_ins & rejected_words) or len(set_output & rejected_words):
            indon = True
            indon_count += 1
            data['instruction_ms'] = mapping[data['instruction']]
            data['output_ms'] = mapping[data['output']]
        else:
            indon = False
            data['instruction_ms'] = None
            data['output_ms'] = None
        
        data['indon'] = indon
        
        fopen_l.write(f'{json.dumps(data)}\n')
        count += 1

100%|██████████████████████████████████| 51714/51714 [00:01<00:00, 32588.10it/s]
